# 협업 필터링을 이용한 영화 추천 엔진
- MovieLens 100k  Data set (10만개)
- 1682개의 영화에 대한 943명의 사용자 평가

- [데이터자료](https://files.grouplens.org/datasets/movielens/) ml-100k.zip

In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("D:/개인공부_머신러닝/ml-100k/u.data", sep='\t', header=None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
df.groupby(['rating'])[['user_id']].count()
# 1점부터 5점까지 점수 확인

,user_id
rating,
1,6110
2,11370
3,27145
4,34174
5,21201


In [10]:
df.groupby(['item_id'])[['user_id']].count().head()
# 영화들에 대한 평가 갯수 확인 가능

,user_id
item_id,
1,452
2,131
3,90
4,209
5,86


In [11]:
#평가 행렬(ratings)의 생성

n_users = df.user_id.unique().shape[0]
n_users
# 실제 유저들의 숫자

943

In [12]:
n_items = df.item_id.unique().shape[0]
n_items
# 아이템의 숫자(영화)

1682

In [13]:
ratings = np.zeros((n_users, n_items))
# np.zeors로 0으로 채워진 행렬을 만드는데, nuser, nitems의 갯수와 맞춤
ratings.shape
# 사용자와 영화수 맞는지 확인, 해당 위치에 추후 평점을 value로 넣을 예정(spars matrix역할)

(943, 1682)

In [14]:
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    #      [사용자, 영화] = 평점 -1을 하는 이유는 인덱스 이기 때문

type(ratings)

numpy.ndarray

In [15]:
ratings.shape

(943, 1682)

In [16]:
ratings
# 각 사용자별 영화의 평점을 열로 저장, 추후 정확한 표현을 위해 sparse matrix 형태로 구현

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [18]:
# 훈련 데이터 테스트 데이터 분리
from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=42)
ratings_train.shape, ratings_test.shape
# 사용자별 분리됨 영화평가한 정보가

((631, 1682), (312, 1682))

In [68]:
# 사용자간 유사도 행렬 생성
from sklearn.metrics.pairwise import cosine_distances #코사인 유사도 cosine_simillarity 간단하게 계산해줌 distances에서 1뺀것과 같음

cosine_distances(ratings_train)
# 각사용자의 영화별 선호도 점수를 유사도점수로 변환함

array([[0.        , 0.63524236, 0.55753769, ..., 0.97989359, 0.66892071,
        0.74361482],
       [0.63524236, 0.        , 0.57364745, ..., 0.93305581, 0.72660686,
        0.77662732],
       [0.55753769, 0.57364745, 0.        , ..., 0.93324244, 0.74575627,
        0.77679874],
       ...,
       [0.97989359, 0.93305581, 0.93324244, ..., 0.        , 0.95146572,
        0.94857492],
       [0.66892071, 0.72660686, 0.74575627, ..., 0.95146572, 0.        ,
        0.8801978 ],
       [0.74361482, 0.77662732, 0.77679874, ..., 0.94857492, 0.8801978 ,
        0.        ]])

In [69]:
# 사용자간 유사도 행렬 생성
from sklearn.metrics.pairwise import cosine_similarity 

cosine_similarity(ratings_train)
# 각사용자의 영화별 선호도 점수를 유사도점수로 변환함

array([[1.        , 0.36475764, 0.44246231, ..., 0.02010641, 0.33107929,
        0.25638518],
       [0.36475764, 1.        , 0.42635255, ..., 0.06694419, 0.27339314,
        0.22337268],
       [0.44246231, 0.42635255, 1.        , ..., 0.06675756, 0.25424373,
        0.22320126],
       ...,
       [0.02010641, 0.06694419, 0.06675756, ..., 1.        , 0.04853428,
        0.05142508],
       [0.33107929, 0.27339314, 0.25424373, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.25638518, 0.22337268, 0.22320126, ..., 0.05142508, 0.1198022 ,
        1.        ]])

In [23]:
cosine_distances(ratings_train).shape

(631, 631)

In [20]:
distances = 1 - cosine_distances(ratings_train)
distances

array([[1.        , 0.36475764, 0.44246231, ..., 0.02010641, 0.33107929,
        0.25638518],
       [0.36475764, 1.        , 0.42635255, ..., 0.06694419, 0.27339314,
        0.22337268],
       [0.44246231, 0.42635255, 1.        , ..., 0.06675756, 0.25424373,
        0.22320126],
       ...,
       [0.02010641, 0.06694419, 0.06675756, ..., 1.        , 0.04853428,
        0.05142508],
       [0.33107929, 0.27339314, 0.25424373, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.25638518, 0.22337268, 0.22320126, ..., 0.05142508, 0.1198022 ,
        1.        ]])

In [21]:
distances.shape

(631, 631)

In [24]:
# 평가 예측 및 모델의 성능 측정
user_pred = distances.dot(ratings_train) / np.array([np.abs(distances).sum(axis=1)]).T
# 사용자 유사도 예측  #np.dot 함수에 train 값 631X631 (631X1682)/ 음수가 있을수 있어 절대값(abs)으로 유사도관련된 합을 열별로 만듬, 크기를 맞추기 위해 전치행렬로 바꿔줌(.T)

In [25]:
from sklearn.metrics import mean_squared_error

In [28]:
def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    # 0이 있을수 있어 0을 제외하고 1차원으로 만듬
    actual = actual[actual.nonzero()].flatten()
    # 0이 있을수 있어 0을 제외하고 1차원으로 만듬
    return mean_squared_error(pred, actual)
    # 그정보를 mean_squred_error에 넣으면 mse값을 얻음

In [29]:
np.sqrt(get_mse(user_pred, ratings_train))
# mse값을 루트 싀우면 RMSE값이 만들어짐

2.8075245308903365

In [30]:
np.sqrt(get_mse(user_pred, ratings_test))
# mse값을 루트 싀우면 RMSE값이 만들어짐

2.9870546415652575

In [ ]:
# 가장 비슷한 사용자를 찾는데 시간이 오래걸리는 단점을 가짐

In [32]:
# 가장 비슷한 n명을 찾는 비지도 방식의 이웃 검색(KNN 과 비슷하나 지도방식임)
from sklearn.neighbors import NearestNeighbors
# 비지도 방식 클래스 NearestNeighbors

k = 5
# 가장비슷한 5개를 찾겠다는 설정
neigh = NearestNeighbors(n_neighbors=k, metric='cosine')
                        # 코사인 유사도로 비슷한걸 찾겟다고 설정, 유사도 계산은 코사인으로

In [33]:
neigh.fit(ratings_train)
# 훈련데이터 적합

NearestNeighbors(metric='cosine')

In [35]:
top_k_distances, top_k_users = neigh.kneighbors(ratings_train, return_distance=True)
# 거리, 유저 =           # kneighbors가 이제 가장 비슷한 5명을 찾아주고 distance(거리)를 반환해줌

In [36]:
top_k_distances.shape, top_k_users.shape

((631, 5), (631, 5))

In [37]:
top_k_users
# 631 명에 해당하는 비슷한 사용자 생성됨

array([[  0, 589, 155,  33, 364],
       [  1, 483, 339, 172, 188],
       [  2, 382, 560, 350, 155],
       ...,
       [628, 258, 242, 229, 494],
       [629, 378, 155, 589, 591],
       [630, 495, 201, 417, 603]], dtype=int64)

In [38]:
top_k_distances
# 사용자별 유사도

array([[0.        , 0.38230161, 0.39990633, 0.40834169, 0.4100445 ],
       [0.        , 0.4625691 , 0.50677921, 0.50811827, 0.50882566],
       [0.        , 0.46538829, 0.48267976, 0.49176259, 0.49265099],
       ...,
       [0.        , 0.5764934 , 0.59340849, 0.64699606, 0.66472075],
       [0.        , 0.60496802, 0.6115226 , 0.62054374, 0.6229481 ],
       [0.        , 0.56320216, 0.60221688, 0.60314589, 0.6400121 ]])

In [39]:
ratings_train.shape

(631, 1682)

In [41]:
# 선택된 n명의 사용자들의 평가 가중치 합을 사용한 예측 및 모델의 성능 측정
user_pred_k = np.zeros(ratings_train.shape)
# 유사도점수의 크기(631x1682)만큼 0으로 채운 빈 변수를 만들어둠

for i in range(ratings_train.shape[0]):
    # 유저의 수 만큼 반복함(631)
    user_pred_k[i, :] = top_k_distances[i].T.dot(ratings_train[top_k_users][i]) / \
    # 예측값은 행, 전체를 바꿈 = 뒤에잇는 값(ratings_train[top_k_users][i])과 크기가 안맞아 전치로 돌림
        np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T

In [42]:
user_pred_k.shape

(631, 1682)

In [43]:
user_pred_k

array([[4.25618269, 2.49082621, 0.71654943, ..., 0.        , 0.        ,
        0.        ],
       [3.74418756, 0.        , 2.48873124, ..., 0.        , 0.        ,
        0.        ],
       [3.22293592, 2.98635211, 2.47648118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.07143091, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.73945823, 2.48622549, 1.76969702, ..., 0.        , 0.        ,
        0.        ],
       [1.95357502, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [44]:
np.sqrt(get_mse(user_pred_k, ratings_train))

2.0922014531938316

In [45]:
np.sqrt(get_mse(user_pred_k, ratings_test))

3.054698791142718

In [ ]:
# 가장 비슷한 5명의 사용자끼리만 비교하도록 개선한 모델

In [46]:
# 영화의 수를 k로 사용해 영화간 유사도 행렬 계산
k = ratings_train.shape[1]
# 영화의 수 k 1682개
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")
                        # 영화의갯수, 유사도계산 cosine 으로 영화간 유사도 구함

In [47]:
neigh.fit(ratings_train.T)
# ratings 는 631 x 1682 므로 T로 전치시켜 크기를 맞춰줌

NearestNeighbors(metric='cosine', n_neighbors=1682)

In [49]:
item_distances, _ = neigh.kneighbors(ratings_train.T, return_distance=True)
# 거리 = 유사도 영화간 유사도 행렬을 구해줌

In [50]:
item_distances

array([[0.        , 0.28112785, 0.30634744, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.34340416, 0.36248159, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.53423537, 0.54586079, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.4452998 , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.0513167 , 0.50763404, ..., 1.        , 1.        ,
        1.        ]])

In [51]:
item_distances.shape
# 영화와 관련된 행렬 생성 확인

(1682, 1682)

In [52]:
item_pred = ratings_train.dot(item_distances) / np.array([np.abs(item_distances).sum(axis=1)])

In [53]:
item_pred.shape

(631, 1682)

In [54]:
item_pred

array([[7.42145170e-17, 2.68346602e-01, 2.73377633e-01, ...,
        5.76694411e-01, 5.91031787e-01, 5.89962397e-01],
       [5.42522608e-17, 1.70392459e-01, 1.73586594e-01, ...,
        3.31747919e-01, 3.39995605e-01, 3.39380431e-01],
       [6.92239530e-17, 3.27208145e-01, 3.32466794e-01, ...,
        6.26634958e-01, 6.42213921e-01, 6.41051924e-01],
       ...,
       [9.98112810e-18, 6.34129164e-02, 6.25449872e-02, ...,
        1.02259215e-01, 1.04801513e-01, 1.04611889e-01],
       [3.78316952e-17, 1.66859982e-01, 1.67810294e-01, ...,
        3.23424495e-01, 3.31465250e-01, 3.30865509e-01],
       [1.86743687e-17, 2.53346239e-02, 2.63412738e-02, ...,
        6.24256837e-02, 6.39776677e-02, 6.38619090e-02]])

In [56]:
np.sqrt(get_mse(item_pred, ratings_train))

3.3784093354963565

In [57]:
np.sqrt(get_mse(item_pred, ratings_test))

3.5098535318342443